In [4]:
import ee
import os
import math
from datetime import datetime
from src.utils import ee_treatments
from src.utils import runs
from src.utils.check_exists import check_exists
from src.utils.yml_params import get_export_params
from src.utils.ee_csv_parser import parse_txt, to_numeric
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'ee_treatments_development'
ee.data.setWorkloadTag(tag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Surface Fuel Updates
There will be GridFire runs for Year 1, Year 10, and Year 20
We have four unique pixel values in our treatment simulation images, the values represent:
1 (years 1-5) 
2 (years 6-10) 
3 (years 11-15)
4 (years 16-20)

We will want to generate newly updated FM40 layers for each GridFire run year, 
updating only those treated pixels valued 1,2, and 4


We will rely on a simplified lookup table to remap those values 

In [29]:
# Plug in treatment scenario ee.Image path
treated_path = "projects/mas-gee/assets/treatment_scenarios/RunID1/SN_WUI_500k_20230623"
treated = ee.Image(treated_path)
baseline_fm40 = ee.Image("projects/pyregence-ee/assets/conus/landfire/fbfm40/LF2022_FBFM40_220")

In [6]:
# test ee blob table encoding working properly
blob = ee.Blob("gs://landfire/LFTFCT_tables/million-acres-LUTs/LANDFIRE fuel model subs - FM40.csv")
table = parse_txt(blob)

# read in the encoded value list as numeric
from_codes = to_numeric(ee.List(table.get("Original FM40_number")))
# read the list of values to remap to as numeric
to_codes = to_numeric(ee.List(table.get("Updated FM40_number")))

print(from_codes.getInfo())
print(to_codes.getInfo())

[91, 92, 93, 98, 99, 101, 102, 103, 104, 121, 122, 123, 141, 142, 143, 144, 145, 146, 147, 161, 162, 163, 165, 181, 182, 183, 184, 185, 186, 187, 188, 189, 201, 202]
[91, 92, 93, 98, 99, 101, 101, 101, 101, 121, 121, 121, 141, 141, 141, 141, 142, 141, 142, 161, 161, 161, 161, 181, 181, 181, 181, 181, 181, 181, 181, 181, 201, 201]


In [21]:
# Keep in mind for WUI YI1 and YI4 share same footprint, so mosaicking burned over the 1's with 4
if 'WUI' in treated_path:
    years = ee.List([4,2,4])
else:
    years = ee.List([1,2,4])

def per_year_update(y):
    treated_mask = treated.eq(ee.Number(y)).selfMask()
    updated_fm40 = baseline_fm40.updateMask(treated_mask).remap(from_codes,to_codes)
    output = updated_fm40.unmask(baseline_fm40)
    return output.rename('new_fm40').set('yearInterval',years.indexOf(y).add(1))

updated_fm40s = years.map(per_year_update)

check_one = ee.Image(ee.List(updated_fm40s).get(0))
print(check_one.get('yearInterval').getInfo())

1


In [19]:
import geemap
Map = geemap.Map()

Map.addLayer(baseline_fm40,{'min':90,'max':205},'baseline')
Map.addLayer(treated.eq(4).selfMask(),{min:0,max:1},'treated Y1')
Map.addLayer(check_one,{'min':90,'max':205},'updated Y1')

Map.centerObject(treated,11)

Map

Map(center=[37.30828177207165, -120.12148367822783], controls=(WidgetControl(options=['position', 'transparent…

In [34]:
# Export each FM40 in list
# California 
ca = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("NAME","California")).first().geometry()
# print(ca.get('NAME').getInfo())
# Map.addLayer(ca.geometry())
# Map
for img in list(range(updated_fm40s.size().getInfo())):
    output = ee.Image(updated_fm40s.get(img))
    year_interval = output.get('yearInterval').getInfo()
    desc = f"UpdateFM40_GFrun{year_interval}"
    output_path = os.path.dirname(treated_path)+"/"+f"FM40_GFrun{year_interval}"
    
    ee_treatments.export_img(output,desc,output_path,ca,30,'EPSG:3310')


Export Started for projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun1
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun2
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun1
